## Import Library

In [27]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import re

# Code

In [28]:
# https://api.mangadex.org/manga?limit=32&offset=32
# https://api.mangadex.org/manga?limit=32&offset=32+32
# https://api.mangadex.org/statistics/manga?manga[]=6f1f3a84-f2e3-4512-93bf-009fd12cfce6
# https://api.mangadex.org/manga/a15dc49f-2512-46f2-bc1e-201be8234ee5?includes[]=artist&includes[]=author&includes[]=cover_art
# https://mangadex.org/covers/a15dc49f-2512-46f2-bc1e-201be8234ee5/c20365e1-285a-4e93-b701-9fd03ff4ae19.jpg

In [29]:
# limit = 32
# offset = limit
# url = 'https://api.mangadex.org/manga?limit=' + str(limit) + '&offset=' + str(offset)
# response = requests.get(url)
# data = response.json()
# page_count = (round(data['total'] / 248))

# for i in range(1, page_count):
#     offset += limit
#     url = 'https://api.mangadex.org/manga?limit=' + str(limit) + '&offset=' + str(offset)

In [30]:
limit = 32
offset = limit
url = 'https://api.mangadex.org/manga?limit=' + str(limit) + '&offset=' + str(offset)
url_stat = 'https://api.mangadex.org/statistics/manga?manga[]='

def getdataurl(url):
    response = requests.get(url)
    if response.status_code == 200:
        json = []
        lan = ['en', 'ja', 'ko', 'ru', 'zh', 'ko-ro', 'zh-hk','es-la']
        data = response.json()
        
        for comic in data['data']:
            url_manga = 'https://api.mangadex.org/manga'
            url_cover = 'https://mangadex.org/covers'
            id = comic['id']
            
            title = next((v for k, v in comic['attributes']['title'].items() if k in lan), None)
            description = next((v for k, v in comic['attributes']['description'].items() if k in lan), None)
            
            alt_titles = []
            for alt_title_dict in comic['attributes'].get('altTitles', []):
                alt_title = next((v for k, v in alt_title_dict.items() if k in lan), None)
                if alt_title:
                    alt_titles.append(alt_title)
            alt_titles = ', '.join(alt_titles)
            
            genres = []
            for tag in comic['attributes'].get('tags', []):
                tag_name = next((v for k, v in tag['attributes']['name'].items() if k in ['en', 'ja', 'ko']), None)
                if tag_name:
                    genres.append(tag_name)
            genres = ', '.join(genres)
        
            released = comic['attributes']['year'] if comic['attributes']['year'] and int(comic['attributes']['year']) else '-'
            
            rate = 0
            response_rate = requests.get(url_stat+id)
            if response_rate.status_code == 200:
                data_stat = response_rate.json()
                rate = data_stat['statistics'][id]['rating']['average']

            author = ''
            artist = ''
            img = ''
            url_manga += '/'+id+'?includes[]=artist&includes[]=author&includes[]=cover_art'
            response_manga = requests.get(url_manga)
            if response_manga.status_code == 200:
                data_manga = response_manga.json()
                for relation in data_manga['data']['relationships']:
                    if relation['type'] == 'author' and 'attributes' in relation and 'name' in relation['attributes']:
                        author = relation['attributes']['name']
                    if relation['type'] == 'artist' and 'attributes' in relation and 'name' in relation['attributes']:
                        artist = relation['attributes']['name']
                    if relation['type'] == 'cover_art' and 'attributes' in relation and 'fileName' in relation['attributes']:
                        img = f"{url_cover}/{id}/{relation['attributes']['fileName']}"

            json.append({
                'title': title,
                'alt_title': alt_titles,
                'type': comic['type'],
                'description': description,
                'genre': genres,
                'author':author,
                'artist':artist,
                'rate': rate,
                'image': img,
                'released': released,
            })
            
        return json

In [31]:
df = []
limit = 32
offset = limit
url = 'https://api.mangadex.org/manga?limit=' + str(limit) + '&offset=' + str(offset)
data = (requests.get(url)).json()
page_count = (round(data['total'] / 248))

# get data manually 300 page's
indexNum = 901 # 701 # 601 # 501 # 401 #315 # 201 # 100 # 0
page_count = 1100 # 901 # 701 # 601 # 501 # 401 # 315 # 201 # 100

for i in range(indexNum, page_count):
    offset += limit
    url = 'https://api.mangadex.org/manga?limit=' + str(limit) + '&offset=' + str(offset)
    getdata = getdataurl(url)
    df = df + getdata

df = pd.DataFrame(df)
df

,title,alt_title,type,description,genre,author,artist,rate,image,released
0,Tonari no Seki no Yatsu ga Souiu Me de Mitekuru,"となりの席のヤツがそういう目で見てくる, La persona sentada a mi l...",manga,"Of course, viewers are excited, but will those...","Romance, Comedy, School Life",mmk,mmk,9.0457,https://mangadex.org/covers/1d25919b-e677-45c1...,2024
1,Stage S,"ステージS, 呼魂蛇使, Этап S",manga,Ukiyo Meguru is in love with his childhood fri...,"Action, Drama, Supernatural",Harikawa Tomoya,Harikawa Tomoya,7.9674,https://mangadex.org/covers/b60fa5bf-c76a-480a...,2022
2,Moriarty the Patriot,"憂国のモリアーティ, Moriarty's Patriotism, Патриотизм М...",manga,"In the late 19th century, Great Britain rules ...","Historical, Psychological, Crime, Drama, Mystery",Takeuchi Ryousuke,Miyoshi Hikaru,9.2619,https://mangadex.org/covers/1d9fad4e-b81a-4498...,2016
3,Home (Ep),홈,manga,Lee Yoonsung and Park Dokyung grew up together...,"Long Strip, Boys' Love, Drama, Web Comic, Full...",Yep,Yep,7.3333,https://mangadex.org/covers/cd850675-5881-4a4f...,2023
4,Weather Forecast Killer,일기예보 살인마,manga,Lee Saehyun fell into a deeply depressed state...,"Long Strip, Boys' Love, Fantasy, Web Comic, Su...",Bori,Bori,5.7692,https://mangadex.org/covers/72290a1b-338e-48cc...,2024
...,...,...,...,...,...,...,...,...,...,...
6363,Koi no Tsuru wa Hanarerarenai,"The Strings of Love Cannot Be Separated, 恋の弦は離...",manga,Yuu practiced archery in his grandfather’s doj...,"Romance, Comedy, Boys' Love, Sports, School Life",Sukeyama,Sukeyama,8.2121,https://mangadex.org/covers/c5e6ba7d-628c-46e4...,2022
6364,Pokémon Special,"Pokémon Adventures, PokéSpecial, ポケットモンスタースペシャ...",manga,Pokémon Adventures follows the story of the or...,"Thriller, Award Winning, Sci-Fi, Monsters, Act...",Hidenori Kusaka,Satoshi Yamamoto,9.0361,https://mangadex.org/covers/47e5c76d-1420-4bfa...,1997
6365,Daikyojin,大巨人,manga,"In a world of towering giants, two witches emb...","Oneshot, Psychological, Long Strip, Adventure,...",Toshiro,Hidano Kentarou,8.0488,https://mangadex.org/covers/0b3e4da7-6221-4044...,2022
6366,"I like you, so I get it.",,manga,None,"Oneshot, Romance, Girls' Love, Drama, Web Comi...",Sal Jiang,Sal Jiang,7.3111,https://mangadex.org/covers/2c34090a-2427-4ff2...,-


In [32]:
df.to_csv('data/mangadex.csv', index=False, encoding='utf-8')